# Evaluation of MLP log-likelihood

Confirm PyTorch and manually coded MLP log-likelihood coincide

In [1]:
## Import packages

import torch
import torch.nn as nn
import torch.nn.functional as F

from eeyore.models.mlp import Hyperparameters, MLP
from eeyore.data import XOR

## Compute MLP log-likelihood using eeyore API version

In [2]:
## Load XOR data

xor = XOR(dtype=torch.float64)
data = xor.data
labels = xor.labels

In [3]:
## Setup MLP model

hparams = Hyperparameters([2, 2, 1])
model = MLP(hparams=hparams, dtype=torch.float64)

In [4]:
## Fix model parameters

theta0 = torch.tensor([1.1, -2.9, -0.4, 0.8, 4.3, 9.2, 4.44, -3.4, 7.2], dtype=torch.float64)
model.set_params(theta0.clone().detach())

In [5]:
## Compute MLP log-likelihood using eeyore API version

result01 = model.log_lik(data, labels)
result01

tensor(-16.0859, dtype=torch.float64, grad_fn=<NegBackward>)

## Compute MLP log-likelihood using Pytorch loss and cross entropy

In [6]:
## Compute model output using Pytorch forward() method

out = model(data)
# out = model.forward(data)
out

tensor([[0.9997],
        [0.9994],
        [0.9997],
        [0.9996]], dtype=torch.float64, grad_fn=<SigmoidBackward>)

In [7]:
## Define logit loss

criterion = nn.BCEWithLogitsLoss(reduction='sum')
loss = criterion(out, labels)

In [8]:
## Define logit function

def logit(p):
    return torch.log(p/(1-p))

In [9]:
## Compute MLP log-likelihood using Pytorch binary_cross_entropy_with_logits

result02a = -F.binary_cross_entropy_with_logits(logit(out), labels, reduction='sum')
result02a

tensor(-16.0859, dtype=torch.float64, grad_fn=<NegBackward>)

In [10]:
## Compute MLP log-likelihood using Pytorch binary_cross_entropy

result02b = -F.binary_cross_entropy(out, labels, reduction='sum')
result02b

tensor(-16.0859, dtype=torch.float64, grad_fn=<NegBackward>)

## Compute MLP log-likelihood manually given Pytorch forward output

In [11]:
## Define sigmoid function

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [12]:
## Define MLP log-likelihood

def loglik(x, y):
    term = y * torch.log(sigmoid(x)) + (1-y) * torch.log(1-sigmoid(x))
    return torch.sum(term)

In [13]:
## Compute MLP log-likelihood manually

result03 = loglik(logit(out), labels)
result03

tensor(-16.0859, dtype=torch.float64, grad_fn=<SumBackward0>)

In [14]:
[p.data.item() for p in [result01, result02a, result02b, result03]]

[-16.08587869723768,
 -16.08587869723768,
 -16.08587869723768,
 -16.08587869723768]